In [1]:
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy.engine import create_engine


In [2]:
#LIMIT = None
LIMIT = 500000
used_cols_with_target = ['item_id','category_name','price','deal_probability']
used_cols_without_target = ['item_id','category_name','price']
target = 'deal_probability'

Create a databas called avito.db and add train.csv into the database

In [3]:
# ONLY NEED TO DO ONCE TO CREATE DATABASE

#df = pd.read_csv('data/train.csv').dropna()
#avito = sqlite3.connect("avito.db")
#df.to_sql('data/train', avito)

#df = pd.read_csv('data/periods_train.csv').dropna()
#avito = sqlite3.connect("avito.db")
#df.to_sql('data/periods_train', avito)

In [4]:
train_pd = pd.read_csv('data/train.csv', usecols=used_cols_with_target)[:LIMIT]
train = train_pd.drop(target,axis = 1).dropna()
train_active = pd.read_csv('data/train_active.csv', usecols=used_cols_without_target)[:LIMIT].dropna()
train_all = train.append(train_active)


In [5]:
price_category_mean = train_all[['price', 'category_name']].groupby('category_name').mean().reset_index()
price_category_std = train_all[['price', 'category_name']].groupby('category_name').std().reset_index()
cate_train_active = pd.merge(price_category_mean,price_category_mean, on='category_name')
cate_train_active.rename(columns={'price_x':'price_mean', 'price_y':'price_std'}, inplace=True)
cate_train_active

,category_name,price_mean,price_std
0,Автомобили,4.041020e+05,4.041020e+05
1,Аквариум,3.395962e+03,3.395962e+03
2,Аудио и видео,1.173186e+05,1.173186e+05
3,Билеты и путешествия,1.781244e+04,1.781244e+04
4,Бытовая техника,7.083943e+03,7.083943e+03
5,Велосипеды,8.377978e+03,8.377978e+03
6,Водный транспорт,3.382900e+05,3.382900e+05
7,Гаражи и машиноместа,2.698687e+05,2.698687e+05
8,Готовый бизнес,4.014545e+06,4.014545e+06
9,Грузовики и спецтехника,7.776433e+05,7.776433e+05


In [6]:
# assign category price mean and std values to each item
#price_category_mean = train_active[['price', 'category_name']].groupby('category_name').mean().reset_index()
#price_category_std = train_active[['price', 'category_name']].groupby('category_name').std().reset_index()

In [7]:
cate_train_active = cate_train_active[['category_name','price_mean','price_std']]

In [8]:
cate_train_active.to_csv('cate_price.csv', index=False)

In [9]:
#cate_train_active['price']['mean']

Calculate price normal

In [10]:
#train = pd.read_csv('data/train.csv', usecols=used_cols)
#train = pd.read_csv('data/train.csv', usecols=used_cols).dropna()
train['category_name'].nunique()
train['price_normal'] = 0
train.head()

,item_id,category_name,price,price_normal
0,b912c3c6a6ad,Товары для детей и игрушки,400.0,0
1,2dac0150717d,Мебель и интерьер,3000.0,0
2,ba83aefab5dc,Аудио и видео,4000.0,0
3,02996f1dd2ea,Товары для детей и игрушки,2200.0,0
4,7c90be56d2ab,Автомобили,40000.0,0


In [11]:
cate_train_active = pd.read_csv('cate_price.csv')
cate_train_active.head()

,category_name,price_mean,price_std
0,Автомобили,404102.037074,404102.037074
1,Аквариум,3395.962146,3395.962146
2,Аудио и видео,117318.628867,117318.628867
3,Билеты и путешествия,17812.435844,17812.435844
4,Бытовая техника,7083.942825,7083.942825


In [12]:
# TEST IF TRAIN MAKES SENSE
#cate_train = train.groupby('category_name').agg({'price': [np.mean, np.std]})
#cate_train

In [13]:
train_pd['cat_price'] = np.nan
train_pd['cat_std'] = np.nan
train_pd['norm_price'] = np.nan

In [14]:
train_pd

,item_id,category_name,price,deal_probability,cat_price,cat_std,norm_price
0,b912c3c6a6ad,Товары для детей и игрушки,400.0,0.12789,NaN,NaN,NaN
1,2dac0150717d,Мебель и интерьер,3000.0,0.00000,NaN,NaN,NaN
2,ba83aefab5dc,Аудио и видео,4000.0,0.43177,NaN,NaN,NaN
3,02996f1dd2ea,Товары для детей и игрушки,2200.0,0.80323,NaN,NaN,NaN
4,7c90be56d2ab,Автомобили,40000.0,0.20797,NaN,NaN,NaN
5,51e0962387f7,Товары для детей и игрушки,1300.0,0.80323,NaN,NaN,NaN
6,c4f260a2b48a,Ремонт и строительство,11000.0,0.00000,NaN,NaN,NaN
7,6b71309d6a8a,"Одежда, обувь, аксессуары",500.0,0.80323,NaN,NaN,NaN
8,c5b969cb63a2,"Одежда, обувь, аксессуары",500.0,0.00000,NaN,NaN,NaN
9,b1570962e68c,Детская одежда и обувь,400.0,0.00000,NaN,NaN,NaN


In [15]:
for x in range (0,len(train_pd)-1):
    train_pd.cat_price[x] =  cate_train_active.price_mean [ cate_train_active.category_name == train_pd.category_name[x] ]
    train_pd.cat_std[x] =  cate_train_active.price_std [ cate_train_active.category_name == train_pd.category_name[x] ] 

/Users/zexuanxu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/zexuanxu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
train_pd['norm_price'] =  (train_pd['price'] - train_pd['cat_price'] ) / train_pd['cat_std'] 

In [17]:
#for i, item in enumerate(train['item_id'][0:]):    
#    for j, cate in enumerate(cate_train_active['category_name']):
#        if train['category_name'][i] == cate:
#            train['price_normal'][i] = (train['price'][i] - cate_train_active['price_mean'][j])/cate_train_active['price_std'][j]             

In [18]:
train_pd

,item_id,category_name,price,deal_probability,cat_price,cat_std,norm_price
0,b912c3c6a6ad,Товары для детей и игрушки,400.0,0.12789,3.878576e+03,3.878576e+03,-0.896869
1,2dac0150717d,Мебель и интерьер,3000.0,0.00000,3.258027e+04,3.258027e+04,-0.907920
2,ba83aefab5dc,Аудио и видео,4000.0,0.43177,1.173186e+05,1.173186e+05,-0.965905
3,02996f1dd2ea,Товары для детей и игрушки,2200.0,0.80323,3.878576e+03,3.878576e+03,-0.432782
4,7c90be56d2ab,Автомобили,40000.0,0.20797,4.041020e+05,4.041020e+05,-0.901015
5,51e0962387f7,Товары для детей и игрушки,1300.0,0.80323,3.878576e+03,3.878576e+03,-0.664825
6,c4f260a2b48a,Ремонт и строительство,11000.0,0.00000,1.979678e+04,1.979678e+04,-0.444354
7,6b71309d6a8a,"Одежда, обувь, аксессуары",500.0,0.80323,7.134283e+03,7.134283e+03,-0.929916
8,c5b969cb63a2,"Одежда, обувь, аксессуары",500.0,0.00000,7.134283e+03,7.134283e+03,-0.929916
9,b1570962e68c,Детская одежда и обувь,400.0,0.00000,1.070896e+04,1.070896e+04,-0.962648


In [19]:
train_pd.to_csv('train_price_normalized.csv')

Select the item_id term only

In [ ]:
#item = pd.read_sql('SELECT item_id,category_name,price FROM train_active WHERE price > 10000', avito)


SELECT all items with price > 10000

In [ ]:
#price = pd.read_sql('SELECT * FROM train WHERE price > 10000 ', avito)
#price

In [ ]:
test = pd.read_csv('data/test.csv', usecols=used_cols_without_target)[:LIMIT]

In [ ]:
test['cat_price'] = np.nan
test['cat_std'] = np.nan
test['norm_price'] = np.nan

In [ ]:
test

In [ ]:
cate_train_active

In [ ]:
for x in range (0,len(test)-1):
    test.cat_price[x] =  cate_train_active.price_mean [ cate_train_active.category_name == test.category_name[x] ]
    test.cat_std[x] =  cate_train_active.price_std [ cate_train_active.category_name == test.category_name[x] ] 

In [ ]:
test.to_csv('test_price_normalized.csv')